# Donnée de stock

Dans les données de stock, il y a un stock inital a date du 5 mai 2017 et un ensemble de stock initial délivré à différent intervals.

Le format de donnée est stocké dans le json suivant:

- https://s3.console.aws.amazon.com/s3/object/calfdata/INPI/TC_1/Stock/dtypes_stock.json

## Documentation

- [Doc INSEE](https://scm.saas.cagip.group.gca/PERNETTH/database/blob/master/Documentation/IMR/Doc_Tech_IMR_Mai_2019_v1.5.1.pdf)
- [etalab](https://github.com/etalab/rncs_worker_api_entreprise)
    - [Module data cleaning](https://github.com/etalab/rncs_worker_api_entreprise/blob/master/clean_csv.bash)
- Path local: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Stock\Stock_initial`
- Path Reseau: `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\stock\2017`

### Variables communes

![image](https://drive.google.com/uc?export=view&id=1laCGFEtxM4bbckNZQEkw1icBjF4pd20-)

### Stock

Le contenu de chaque fichier est disponible [ici](https://scm.saas.cagip.group.gca/PERNETTH/database/tree/master/INPI/TC_1)

Les données brutes sont répertoriées dans des fichiers zip composées de 7 fichiers:

- Actes
- Comptes Annuels
- ETS
- OBS
- PM
- PP
- REP

### Donnees de flux 

- Path local: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux`
- Path Reseau: `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux`

Les PP en local sont disponibles ici: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\PP`

name csv : `0101_1_20170512_112544_4_PP_EVT`
    
    - code greffe: 0101
    - numéro transmission: 1
    - Date: 2017-05-12
    - heures: 11h25m44s
    - Type: PP
    - Evenement: Oui

**FICHIERS TRANSMIS EN CAS DE MISE A JOUR D’UN DOSSIER (EVENEMENT)**

En cas de mise à jour d’un dossier suite à un événement (modification, radiation), les fichiers transmis ont une structure identique aux fichiers créés à l’immatriculation avec la présence de 2 champs spécifiques : la date de l’événement (Date_Greffe) et le libellé de l’événement (Libelle_Evt).

Dans ces cas, 6 types de fichiers supplémentaires, numérotés, sont transmis correspondant à :
* Evénements modifiant ou complétant les dossiers d’immatriculation des personnes morales (2) ou physiques (4)
* Evénements modifiant ou complétant les informations relatives aux représentants (6) ou aux établissements (9)
* Evénements supprimant des représentants (7 – Représentant partant) ou des établissements (10 – Etablissement supprimé)

![](https://drive.google.com/uc?export=view&id=1FVEGNqogl1NxB84BtdF4TQztCXjmtpyo)

Attention, il peut arriver que le même dossier fasse l’objet de plusieurs événements (création et modification) dans la même transmission. Il est impératif d’intégrer les événements dans l’ordre d’apparition.

Les csv sont classés de la manière suivante:

- Module (ETS, PP, PM, etc)
    - NEW: New event
    - EVT: Update event
    
## REP

Doc REP: https://scm.saas.cagip.group.gca/PERNETTH/database/blob/master/Documentation/IMR/Doc_Tech_IMR_Mai_2019_v1.5.1.pdf
- page 30

Attention, certains champs sont remplis selon le type de la société. Ex SIREN -> Si type =PM

Pourquoi deux SIREN?

Le deuxième siren correspond a celui de la PM qui detient l'entreprise.

Ex: Peugeot a plusieurs détenteur. Le détenteur possède un siren, il sera affiché dans siren.1 uniquement si détenteur est une PM

In [ ]:
#!pip install tqdm

In [ ]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.inpi as inpi
#from tqdm.notebook import tqdm
import tqdm
%load_ext autoreload
%autoreload 2

## Batch Stock aggregation csv

In [ ]:
#from sagemaker import get_execution_role
#role = get_execution_role() 
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'
agg_datasock = inpi.inpiStock(instance_aws, bucket)
agg_datasock.uploadFileBucket(pathfile = 'INPI/TC_1/Stock/dtypes_stock.json')
with open('dtypes_stock.json', 'r') as f:
    datastore = json.load(f)

In [ ]:
for key, value in tqdm.tqdm(datastore['option'].items()): 
    if key not in ['PP', 'PM', 'ETS']:
        print(key)
        dtype = value["dtype"]
        parse_dates = value['parse_dates']
        
        for p in [True, False]:
            agg_datasock.appendInpiStock(
                key,
                dtype,
                parse_dates,
                partiel = p,
                return_frame = False)

## Batch flux aggregation csv

In [ ]:
for key, value in tqdm.tqdm(datastore['option'].items()): 
    if key not in ['PP', 'PM', 'ETS']:
        print(key)
        dtype = value["dtype"]
        parse_dates = value['parse_dates']
        agg_datasock.appendInpiFlux(
            option_extract =key,
            new = True,
            year = 2017,
            dtype = dtype,
            parse_date = parse_dates,
            return_frame = True)

In [ ]:
import os
os.remove('dtypes_stock.json')

In [ ]:
#agg_datasock.appendInpiStock('PP',
#                             dtype,
#                             parse_dates)

In [ ]:
#agg_datasock.appendInpiStock('PP',
#                                    dtype,
#                                    parse_dates,
#                                    partiel = True)